In [18]:
import pycolmap as pc
from pathlib import Path
import sys
sys.path.append('/src/app')
sys.path.append('/src/')
sys.path.append('src')
from place_utils import localize_places

In [2]:
dset = 'canteen-20221115'
output = 'canteen-20221115-sift'
datasetz_path = Path(f'/src/matcher_engine/HierarchicalLocalization/outputs/{output}')
feature_conf = "superpoint_aachen"
matcher_conf = "superglue"

In [3]:
sfm = datasetz_path / 'sfm_superpoint+superglue'
sfm

PosixPath('/src/matcher_engine/HierarchicalLocalization/outputs/canteen-20221115-sift/sfm_superpoint+superglue')

In [4]:
recon = pc.Reconstruction()

In [16]:
import numpy as np
import cv2
import tqdm
import os
from IPython.display import clear_output

# Clear the output of the current cell
clear_output()

def get_qt_vector(recon):
    res = {}
    for k,image in recon.images.items():
        qtvec = np.concatenate((image.qvec,image.tvec), axis=0)
        res[image.name] = qtvec
    return res

def matcher_localize(image_paths, image_folder_path, dataset_name, num=20, rank=5, cvtColor=None, limit=9999):
    poses = {}
    count = 0
    for image_path in tqdm.tqdm(image_paths):
        full_image_path = os.path.join(image_folder_path, image_path)
        args = {"name": dataset_name,
                "input": full_image_path,
                "num": num,
                "rank": rank,
                "feature_conf":feature_conf,
                "matcher_conf":matcher_conf,}
        print(args)
        image = cv2.imread(full_image_path)
        if cvtColor: image = cv2.cvtColor(image, cvtColor)
        queries = [(os.path.basename(full_image_path), image)]
        pose = localize_places(queries, args)
        poses[image_path] = pose
        count+=1
        if count == limit: break
        clear_output()
    return poses

def set_query_poses_to_model(recon, poses):
    """
    use output from matcher localize
    """
    for i, image in recon.images.items():
        image_name = image.name
        q = poses[image_name][0]['orientation']
        t = poses[image_name][0]['position']
        old_qvec = recon.images[i].qvec
        old_tvec = recon.images[i].tvec
        recon.images[i].tvec = np.array([t['x'], t['y'], t['z']])
        recon.images[i].qvec = np.array([q['w'], q['x'], q['y'], q['z']])
    print("set new camera poses success!")

In [6]:
{'name': 'canteen-20221115', 'input': '/src/matcher_engine/HierarchicalLocalization/datasets/canteen-20221115/images/db/p001-d3-f00034.jpg', 'num': 20, 'rank': 5}
{'name': 'canteen-20221115', 'input': '/src/matcher_engine/HierarchicalLocalization/datasets/canteen-20221115/images/db/p012-d7-f00286.jpg', 'num': 20, 'rank': 5}

{'name': 'canteen-20221115',
 'input': '/src/matcher_engine/HierarchicalLocalization/datasets/canteen-20221115/images/db/p012-d7-f00286.jpg',
 'num': 20,
 'rank': 5}

In [7]:
recon.read(str((sfm)))

In [8]:
print(recon.summary())

Reconstruction:
	num_reg_images = 16
	num_cameras = 16
	num_points3D = 1493
	num_observations = 3661
	mean_track_length = 2.45211
	mean_observations_per_image = 228.812
	mean_reprojection_error = 0.717112


In [9]:
qtvecs = get_qt_vector(recon)

In [10]:
print(recon.images[1].qvec)
print(recon.images[1].tvec)

KeyError: ''

In [11]:
%cd /src

/src


In [13]:
image_paths

['db/p012-d4-f00277.jpg',
 'db/p011-d5-f00256.jpg',
 'db/p011-d4-f00253.jpg',
 'db/p011-d3-f00250.jpg',
 'db/p010-d7-f00238.jpg',
 'db/p011-d7-f00262.jpg',
 'db/p009-d6-f00307.jpg',
 'db/p012-d6-f00283.jpg',
 'db/p010-d5-f00232.jpg',
 'db/p011-d6-f00259.jpg',
 'db/p009-d5-f00304.jpg',
 'db/p012-d5-f00280.jpg',
 'db/p010-d4-f00229.jpg',
 'db/p009-d7-f00310.jpg',
 'db/p012-d7-f00286.jpg',
 'db/p010-d6-f00235.jpg']

In [17]:
image_paths = list(qtvecs.keys())

dataset_name = dset
num = 100
image_folder_paths = f'/src/matcher_engine/HierarchicalLocalization/datasets/{dset}/images'
query_poses = matcher_localize(image_paths, image_folder_paths, dataset_name, cvtColor=None, num=num)

  0%|          | 0/16 [00:00<?, ?it/s]

{'name': 'canteen-20221115', 'input': '/src/matcher_engine/HierarchicalLocalization/datasets/canteen-20221115/images/db/p012-d4-f00277.jpg', 'num': 100, 'rank': 5, 'feature_conf': 'superpoint_aachen', 'matcher_conf': 'superglue'}


FileNotFoundError: [Errno 2] No such file or directory: 'matcher_engine/HierarchicalLocalization/datasets/canteen-20221115/images/query'

In [ ]:
query_poses

In [68]:
import pickle 

#nt no translation in get pose ของพีมั่ม
#nip no inverse pose เหมือน get_pose จะ return แต่ inverse pose เลยลบ inverse pose ทิ้ง 
with open("/src/matcher_engine/HierarchicalLocalization/error_analyze/query_poses_"+dataset_name+"_nt_nip.pkl", "wb") as f:
    pickle.dump(query_poses, f)

In [81]:
set_query_poses_to_model(recon, query_poses)

set new camera poses success!


In [70]:
print(recon.images[1].qvec)
print(recon.images[1].tvec)

[0.80069733 0.00605782 0.46331895 0.3797139 ]
[-0.08557794 -0.96359958  3.32017164]


In [71]:
%cd /src/matcher_engine/HierarchicalLocalization/error_analyze

/src/matcher_engine/HierarchicalLocalization/error_analyze


In [82]:
os.makedirs(dataset_name+"_nt_nip", exist_ok=False)
recon.write(dataset_name+"_nt_nip")

In [79]:
num=1
recon = pc.Reconstruction()
recon.read(str((sfm)))
print(recon.images[num].qvec)
print(recon.images[num].tvec)
set_query_poses_to_model(recon, query_poses)
print(recon.images[num].qvec)
print(recon.images[num].tvec)

[0.80054002 0.00632992 0.46333653 0.38001957]
[-0.0854075  -0.96382379  3.33234693]
set new camera poses success!
[0.80069733 0.00605782 0.46331895 0.3797139 ]
[-0.08557794 -0.96359958  3.32017164]


In [54]:
query_poses['db/IMG_0001.JPG']

[{'filename': 'query_2022-12-23T08:50:00.875571.jpg',
  'position': {'x': 3.0636544893579325,
   'y': -0.5663324126792799,
   'z': -1.500891802639155},
  'orientation': {'w': 0.8006973257870776,
   'x': -0.006057820514290533,
   'y': -0.46331895222044445,
   'z': -0.3797138972982022},
  'original_filename': 'IMG_0001.JPG',
  'frame_id': 'cupid'}]

In [12]:
recon.read('/src/matcher_engine/HierarchicalLocalization/error_analyze/canteen-20221115')

In [35]:
from tqdm.notebook import tqdm
import numpy as np

In [43]:
path = "/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/isl2-20221101/weight0/localize_dbqr0"
sfm_model = '/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/isl2-20221101/weight0/map0'
dataset_name = 'isl2-20221101'

In [37]:
recon = pc.Reconstruction()
recon.read(sfm_model)

In [38]:
print(recon.images[1].qvec)
print(recon.images[1].tvec)

[ 6.97996708e-01  7.16094951e-01 -2.93272366e-03  1.29027747e-04]
[ 0.03344537 -0.01407832  0.02043296]


In [39]:
def set_query_poses_to_model(recon, pose_path):
    """
    use output from matcher localize
    """
    raw_txt = open(pose_path, 'r').read().split("\n")[:-1]
    dic = {i.split(",")[0]:i.split(",")[1:] for i in raw_txt}
    for i, image in recon.images.items():
        image_name = image.name
        txt = dic[image_name]
        t = txt
        # q = poses[image_name][0]['orientation']
        # t = poses[image_name][0]['position']
        # old_qvec = recon.images[i].qvec
        # old_tvec = recon.images[i].tvec
        recon.images[i].tvec = np.array([t[0], t[1], t[2]])
        recon.images[i].qvec = np.array([t[3],t[4],t[5],t[6]])
        print("set",end="")
    print("set new camera poses success!")

set_query_poses_to_model(recon, path)

setsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetsetset new camera poses success!


In [40]:
print(recon.images[1].qvec)
print(recon.images[1].tvec)

[ 0.69764183  0.71643542 -0.00383728  0.0012035 ]
[ 0.04027274 -0.01488617  0.01654028]


In [41]:
%cd /src/matcher_engine/HierarchicalLocalization/error_analyze

/src/matcher_engine/HierarchicalLocalization/error_analyze


In [46]:
import os
os.makedirs(dataset_name+"_report")
recon.write(dataset_name+"_report")